## Knowledge Distillation for Question Answering

__Teacher__: `XLM-Roberta-Large`

__Student__: `Multilingual-MiniLM-L12-H384`

In [1]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cuda


In [2]:
from huggingface_hub import notebook_login
notebook_login()

## Models

In [3]:
from transformers import (AutoModelForQuestionAnswering, AutoTokenizer,
                          Trainer, TrainingArguments)

import collections

In [4]:
batch_size = 4
pretraining_language = 'en'

In [5]:
teacher = AutoModelForQuestionAnswering.from_pretrained("deepset/xlm-roberta-base-squad2")
teacher_tokenizer = AutoTokenizer.from_pretrained("deepset/xlm-roberta-base-squad2")


student_ckpt = f"subhasisj/{pretraining_language}-TAPT-MLM-MiniLM"
student_tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")

In [6]:
# Tokenize text and infer from context using QA model
question = "How much music can this hold?"
context = """An MP3 is about 1 MB/minute, so about 6000 hours depending on \
file size."""
inputs = teacher_tokenizer(question, context, return_tensors="pt")
print(teacher_tokenizer.decode(inputs["input_ids"][0]))

<s> How much music can this hold?</s></s> An MP3 is about 1 MB/minute, so about 6000 hours depending on file size.</s>


In [7]:
with torch.no_grad():
    outputs = teacher(**inputs)
print(outputs)

QuestionAnsweringModelOutput(loss=None, start_logits=tensor([[ 1.9587, -5.5965, -6.6265, -6.0578, -6.8384, -6.3983, -5.9162, -7.4403,
         -7.1065, -4.7744,  1.4149,  1.2864, -2.6718, -3.2817,  2.4467,  4.2245,
          0.0827, -3.4912, -3.3558, -4.2590, -4.9737, -1.7109,  3.4588,  2.9234,
         -1.7378, -2.4887, -4.4600, -4.0690, -0.7915, -3.4963, -4.5764, -4.2698]]), end_logits=tensor([[ 1.6167, -7.2887, -5.5725, -4.7752, -7.1230, -6.9200, -5.6986, -6.4568,
         -6.4080, -5.3959, -4.1854, -3.4914,  0.5187, -4.4616, -2.8849, -0.9189,
          1.9932, -2.2891, -1.3932,  3.2144,  0.5945, -2.3245, -3.3690,  0.4770,
          4.4351, -3.9249, -1.0726, -3.9744, -2.8195,  0.6834,  2.3187, -3.3167]]), hidden_states=None, attentions=None)


## Data

In [13]:
import numpy as np
import pandas as pd
import json

def squad_json_to_dataframe_train(input_file_path, record_path = ['data','paragraphs','qas','answers'],
                           verbose = 1):
    """
    input_file_path: path to the squad json file.
    record_path: path to deepest level in json file default value is
    ['data','paragraphs','qas','answers']
    verbose: 0 to suppress it default is 1
    """
    if verbose:
        print("Reading the json file")    
    file = json.loads(open(input_file_path).read())
    if verbose:
        print("processing...")
    # parsing different level's in the json file
    js = pd.io.json.json_normalize(file , record_path )
    m = pd.io.json.json_normalize(file, record_path[:-1] )
    r = pd.io.json.json_normalize(file,record_path[:-2])
    
    #combining it into single dataframe
    idx = np.repeat(r['context'].values, r.qas.str.len())
    ndx  = np.repeat(m['id'].values,m['answers'].str.len())
    m['context'] = idx
    js['q_idx'] = ndx
    main = pd.concat([ m[['id','question','context']].set_index('id'),js.set_index('q_idx')],1,sort=False).reset_index()
    main['c_id'] = main['context'].factorize()[0]
    if verbose:
        print("shape of the dataframe is {}".format(main.shape))
        print("Done")
    return main

In [14]:
# training data
language = pretraining_language
input_file_path = f'../SQuAD/translate-train/squad.translate.train.en-{language}.json'
record_path = ['data','paragraphs','qas','answers']
train = squad_json_to_dataframe_train(input_file_path=input_file_path,record_path=record_path,verbose=0)

C:\Users\Subha\AppData\Local\Temp\ipykernel_23172\3870843590.py:19: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  js = pd.io.json.json_normalize(file , record_path )
C:\Users\Subha\AppData\Local\Temp\ipykernel_23172\3870843590.py:20: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  m = pd.io.json.json_normalize(file, record_path[:-1] )
C:\Users\Subha\AppData\Local\Temp\ipykernel_23172\3870843590.py:21: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  r = pd.io.json.json_normalize(file,record_path[:-2])
C:\Users\Subha\AppData\Local\Temp\ipykernel_23172\3870843590.py:28: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  main = pd.concat([ m[['id','question','context']].set_index('id'),js.set_index('q_idx')],1,sort=False).reset_index()


In [15]:
def get_answers(x):
    start = x[0]
    text = x[1]
    return {
        'answer_start': [start],
        'text': [text]
    }

train['answers'] = train[['answer_start', 'text']].apply(get_answers, axis=1)
pd.set_option('display.max_colwidth',None)
train.head(1)

,index,question,context,answer_start,text,c_id,answers
0,56db2120e7c41114004b4dce,Wer war der Präsident der alternativen Programmierung bei Fox?,"Die 8. Staffel wurde am 13. Januar 2009 uraufgeführt. Mike Darnell, der Präsident der alternativen Programmiersoftware von Fox, erklärte, die Saison werde sich mehr auf das persönliche Leben der Teilnehmer konzentrieren. Viel frühes Augenmerk in der Show konzentrierte sich daher auf die Witwerschaft von Danny Gokey.",54,Mike Darnell,0,"{'answer_start': [54], 'text': ['Mike Darnell']}"


In [16]:
# Validation data
input_file_path = f'../SQuAD/translate-dev/squad.translate.dev.en-{language}.json'
record_path = ['data','paragraphs','qas','answers']
dev = squad_json_to_dataframe_train(input_file_path=input_file_path,record_path=record_path,verbose=0)
dev['answers'] = dev[['answer_start', 'text']].apply(get_answers, axis=1)
pd.set_option('display.max_colwidth',None)
dev.head(1)

C:\Users\Subha\AppData\Local\Temp\ipykernel_23172\3870843590.py:19: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  js = pd.io.json.json_normalize(file , record_path )
C:\Users\Subha\AppData\Local\Temp\ipykernel_23172\3870843590.py:20: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  m = pd.io.json.json_normalize(file, record_path[:-1] )
C:\Users\Subha\AppData\Local\Temp\ipykernel_23172\3870843590.py:21: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead.
  r = pd.io.json.json_normalize(file,record_path[:-2])
C:\Users\Subha\AppData\Local\Temp\ipykernel_23172\3870843590.py:28: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  main = pd.concat([ m[['id','question','context']].set_index('id'),js.set_index('q_idx')],1,sort=False).reset_index()


,index,question,context,answer_start,text,c_id,answers
0,5730a951069b531400832213,"Jules Ferry dachte, dass die ""höheren Rassen"" eine Pflicht zu was haben?","Es wurde zu einer moralischen Rechtfertigung, die Welt durch das Christentum und die französische Kultur auf französische Standards zu bringen. Der führende Exponent des Kolonialismus, Jules Ferry, erklärte 1884, Frankreich habe eine zivilisierende Mission: ""Die höheren Rassen haben ein Recht auf die niederen Rassen, sie haben die Pflicht, die Untergebenen zu zivilisieren"". Es wurden volle Bürgerrechte - ""Assimilation"" - angeboten, obwohl die Assimilation in Wirklichkeit immer am Horizont lag. Im Gegensatz zu Großbritannien schickte Frankreich eine kleine Anzahl von Siedlern in seine Kolonien, mit der einzigen bemerkenswerten Ausnahme in Algerien, wo die französischen Siedler dennoch immer eine kleine Minderheit blieben.",342,die Untergebenen zu zivilisieren,0,"{'answer_start': [342], 'text': ['die Untergebenen zu zivilisieren']}"


### Create Datasets

In [17]:
from datasets import Dataset,DatasetDict


tds = Dataset.from_pandas(train)
vds = Dataset.from_pandas(dev)


squad = DatasetDict()

squad['train'] = tds
squad['validation'] = vds

In [8]:
from datasets import load_dataset
squad = load_dataset("squad")

Reusing dataset squad (C:\Users\Subha\.cache\huggingface\datasets\squad\plain_text\1.0.0\d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453)


  0%|          | 0/2 [00:00<?, ?it/s]

In [9]:
pad_on_right = student_tokenizer.padding_side == "right"
max_length = 384 # The maximum length of a feature (question and context)
doc_stride = 128 # The authorized overlap between two part of the context when splitting it is needed.
def prepare_train_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = student_tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # The offset mappings will give us a map from token to character position in the original context. This will
    # help us compute the start_positions and end_positions.
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Let's label those examples!
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(student_tokenizer.cls_token_id)

        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        # If no answers are given, set the cls_index as answer.
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                token_end_index -= 1

            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

def prepare_validation_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    # We keep the example_id that gave us this feature and we will store the offset mappings.
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1 if pad_on_right else 0

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])

        # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token
        # position is part of the context or not.
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples

def postprocess_qa_predictions(
    examples, features, raw_predictions, n_best_size=20, max_answer_length=30
):
    all_start_logits, all_end_logits = raw_predictions
    # Build a map example to its corresponding features.
    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)

    # The dictionaries we have to fill.
    predictions = collections.OrderedDict()

    # Logging.
    print(
        f"Post-processing {len(examples)} example predictions split into {len(features)} features."
    )

    # Let's loop over all the examples!
    for example_index, example in enumerate(tqdm(examples)):
        # Those are the indices of the features associated to the current example.
        feature_indices = features_per_example[example_index]

        min_null_score = None  # Only used if squad_v2 is True.
        valid_answers = []

        context = example["context"]
        # Looping through all the features associated to the current example.
        for feature_index in feature_indices:
            # We grab the predictions of the model for this feature.
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
            # This is what will allow us to map some the positions in our logits to span of texts in the original
            # context.
            offset_mapping = features[feature_index]["offset_mapping"]

            # Update minimum null prediction.
            cls_index = features[feature_index]["input_ids"].index(
                tokenizer.cls_token_id
            )
            feature_null_score = start_logits[cls_index] + end_logits[cls_index]
            if min_null_score is None or min_null_score < feature_null_score:
                min_null_score = feature_null_score

            # Go through all possibilities for the `n_best_size` greater start and end logits.
            start_indexes = np.argsort(start_logits)[
                -1 : -n_best_size - 1 : -1
            ].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
                    # to part of the input_ids that are not in the context.
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or offset_mapping[end_index] is None
                        or offset_mapping[start_index] == []
                        or offset_mapping[end_index] == []
                    ):
                        continue
                    # Don't consider answers with a length that is either < 0 or > max_answer_length.
                    if (
                        end_index < start_index
                        or end_index - start_index + 1 > max_answer_length
                    ):
                        continue

                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]
                    valid_answers.append(
                        {
                            "score": start_logits[start_index] + end_logits[end_index],
                            "text": context[start_char:end_char],
                        }
                    )

        if len(valid_answers) > 0:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[
                0
            ]
        else:
            # In the very rare edge case we have not a single non-null prediction, we create a fake prediction to avoid
            # failure.
            best_answer = {"text": "", "score": 0.0}

        # Let's pick our final answer: the best one or the null answer (only for squad_v2)
        # if not squad_v2:
        # predictions[example["id"]] = best_answer["text"]
        # else:
        answer = best_answer["text"] if best_answer["score"] > min_null_score else ""
        predictions[example["id"]] = answer

    return predictions

In [10]:
squad = squad.map(prepare_train_features,batched=True,remove_columns=squad["train"].column_names)

Loading cached processed dataset at C:\Users\Subha\.cache\huggingface\datasets\squad\plain_text\1.0.0\d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453\cache-df71c0ea18dee511.arrow


  0%|          | 0/11 [00:00<?, ?ba/s]

## Training

In [11]:
import torch.nn as nn
import torch.nn.functional as F

class DistillationTrainingArguments(TrainingArguments):
    def __init__(self, *args, alpha=0.5, temperature=2.0, **kwargs):
        super().__init__(*args, **kwargs)
        self.alpha = alpha
        self.temperature = temperature

class DistillationTrainer(Trainer):
    def __init__(self, *args, teacher_model=None, **kwargs):
        super().__init__(*args, **kwargs)
        self.teacher_model = teacher_model

    def compute_loss(self, model, inputs,return_outputs=False):
        outputs_stu = model(**inputs)
        # Extract cross-entropy loss and logits from student
        loss_ce = outputs_stu.loss
        # Extract logits from teacher
        with torch.no_grad():
            outputs_tea = self.teacher_model(**inputs)
            start_logits_tea = outputs_tea.start_logits
            end_logits_tea = outputs_tea.end_logits
        # Soften probabilities and compute distillation loss
        loss_fct = nn.KLDivLoss(reduction="batchmean")
        
        loss_start = loss_fct(
                    nn.functional.log_softmax(outputs_stu.start_logits / self.args.temperature, dim=-1),
                    nn.functional.softmax(start_logits_tea / self.args.temperature, dim=-1),
                ) * (self.args.temperature**2)
        loss_end = loss_fct(
            nn.functional.log_softmax(outputs_stu.end_logits / self.args.temperature, dim=-1),
            nn.functional.softmax(end_logits_tea / self.args.temperature, dim=-1),
        ) * (self.args.temperature**2)
        loss_kd = (loss_start + loss_end) / 2.0

        loss = self.args.alpha * loss_ce + (1 - self.args.alpha) * loss_kd

        return loss

In [12]:
from transformers import default_data_collator


from tqdm.auto import tqdm


def compute_metrics(start_logits, end_logits, features, examples):
    example_to_features = collections.defaultdict(list)
    for idx, feature in enumerate(features):
        example_to_features[feature["example_id"]].append(idx)

    predicted_answers = []
    for example in tqdm(examples):
        example_id = example["id"]
        context = example["context"]
        answers = []

        # Loop through all features associated with that example
        for feature_index in example_to_features[example_id]:
            start_logit = start_logits[feature_index]
            end_logit = end_logits[feature_index]
            offsets = features[feature_index]["offset_mapping"]

            start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
            end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Skip answers that are not fully in the context
                    if offsets[start_index] is None or offsets[end_index] is None:
                        continue
                    # Skip answers with a length that is either < 0 or > max_answer_length
                    if (
                        end_index < start_index
                        or end_index - start_index + 1 > max_answer_length
                    ):
                        continue

                    answer = {
                        "text": context[offsets[start_index][0] : offsets[end_index][1]],
                        "logit_score": start_logit[start_index] + end_logit[end_index],
                    }
                    answers.append(answer)

        # Select the answer with the best score
        if len(answers) > 0:
            best_answer = max(answers, key=lambda x: x["logit_score"])
            predicted_answers.append(
                {"id": example_id, "prediction_text": best_answer["text"]}
            )
        else:
            predicted_answers.append({"id": example_id, "prediction_text": ""})

    theoretical_answers = [{"id": ex["id"], "answers": ex["answers"]} for ex in examples]
    return metric.compute(predictions=predicted_answers, references=theoretical_answers)
    
def student_init():
    return AutoModelForQuestionAnswering.from_pretrained(
        f"subhasisj/{pretraining_language}-TAPT-MLM-MiniLM"
    ).to(device)


student_training_args = DistillationTrainingArguments(
    output_dir = f"./{pretraining_language}-kd-XLM-minilmv2-{batch_size}",
    # evaluation_strategy="epoch",
    save_strategy = "epoch",
    learning_rate = 3e-5,
    num_train_epochs=5,
    per_device_train_batch_size=batch_size,
    # per_device_eval_batch_size=batch_size,
    temperature = 2.0,
    alpha=0.5,
    warmup_ratio = 0.1,
    fp16=True,
    gradient_accumulation_steps = 8,
    weight_decay=0.01,
    push_to_hub=True,
    save_total_limit=1,
    # load_best_model_at_end=True
)

teacher.to(device)


data_collator = default_data_collator

distil_trainer = DistillationTrainer(
    model_init=student_init,
    teacher_model=teacher,
    args=student_training_args,
    train_dataset=squad["train"],
    # eval_dataset=squad["validation"],
    data_collator=data_collator,
    tokenizer=student_tokenizer,
    # compute_metrics=compute_metrics,
)


loading configuration file https://huggingface.co/subhasisj/en-TAPT-MLM-MiniLM/resolve/main/config.json from cache at C:\Users\Subha/.cache\huggingface\transformers\8b4816e9caacf92de112b26ffc9f52aab76d3d653090de57337d2725e12916fa.e10bd0b72b4099c977ef126c1372e211950889239eb7513aba618c575d204753
Model config BertConfig {
  "_name_or_path": "subhasisj/en-TAPT-MLM-MiniLM",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 384,
  "initializer_range": 0.02,
  "intermediate_size": 1536,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "tokenizer_class": "XLMRobertaTokenizer",
  "torch_dtype": "float32",
  "transformers_version": "4.18.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 25003

Downloading:   0%|          | 0.00/450M [00:00<?, ?B/s]

storing https://huggingface.co/subhasisj/en-TAPT-MLM-MiniLM/resolve/main/pytorch_model.bin in cache at C:\Users\Subha/.cache\huggingface\transformers\a1e75a81f215fe95befe29376d314de3ced96c4b880ff2f7e313a5ae41d445b2.adfa05f5140e4af7ae4b32ff7ceb4131265c3e4aa79a67964d941e6df193cc7b
creating metadata file for C:\Users\Subha/.cache\huggingface\transformers\a1e75a81f215fe95befe29376d314de3ced96c4b880ff2f7e313a5ae41d445b2.adfa05f5140e4af7ae4b32ff7ceb4131265c3e4aa79a67964d941e6df193cc7b
loading weights file https://huggingface.co/subhasisj/en-TAPT-MLM-MiniLM/resolve/main/pytorch_model.bin from cache at C:\Users\Subha/.cache\huggingface\transformers\a1e75a81f215fe95befe29376d314de3ced96c4b880ff2f7e313a5ae41d445b2.adfa05f5140e4af7ae4b32ff7ceb4131265c3e4aa79a67964d941e6df193cc7b
Some weights of the model checkpoint at subhasisj/en-TAPT-MLM-MiniLM were not used when initializing BertForQuestionAnswering: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.pred

In [13]:
distil_trainer.train()

loading configuration file https://huggingface.co/subhasisj/en-TAPT-MLM-MiniLM/resolve/main/config.json from cache at C:\Users\Subha/.cache\huggingface\transformers\8b4816e9caacf92de112b26ffc9f52aab76d3d653090de57337d2725e12916fa.e10bd0b72b4099c977ef126c1372e211950889239eb7513aba618c575d204753
Model config BertConfig {
  "_name_or_path": "subhasisj/en-TAPT-MLM-MiniLM",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 384,
  "initializer_range": 0.02,
  "intermediate_size": 1536,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "tokenizer_class": "XLMRobertaTokenizer",
  "torch_dtype": "float32",
  "transformers_version": "4.18.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 25003

  0%|          | 0/14000 [00:00<?, ?it/s]

{'loss': 6.5664, 'learning_rate': 1.0714285714285714e-05, 'epoch': 0.18}
{'loss': 4.2442, 'learning_rate': 2.1428571428571428e-05, 'epoch': 0.36}
{'loss': 2.8929, 'learning_rate': 2.9764285714285715e-05, 'epoch': 0.54}
{'loss': 1.778, 'learning_rate': 2.8573809523809524e-05, 'epoch': 0.71}
{'loss': 1.2159, 'learning_rate': 2.7385714285714285e-05, 'epoch': 0.89}


Saving model checkpoint to ./en-kd-XLM-minilmv2-4\checkpoint-2800
Configuration saved in ./en-kd-XLM-minilmv2-4\checkpoint-2800\config.json
Model weights saved in ./en-kd-XLM-minilmv2-4\checkpoint-2800\pytorch_model.bin
tokenizer config file saved in ./en-kd-XLM-minilmv2-4\checkpoint-2800\tokenizer_config.json
Special tokens file saved in ./en-kd-XLM-minilmv2-4\checkpoint-2800\special_tokens_map.json
tokenizer config file saved in ./en-kd-XLM-minilmv2-4\tokenizer_config.json
Special tokens file saved in ./en-kd-XLM-minilmv2-4\special_tokens_map.json
Adding files tracked by Git LFS: ['tokenizer.json']. This may take a bit of time if the files are large.


{'loss': 0.9848, 'learning_rate': 2.6195238095238094e-05, 'epoch': 1.07}
{'loss': 0.8312, 'learning_rate': 2.5004761904761907e-05, 'epoch': 1.25}
{'loss': 0.7867, 'learning_rate': 2.3814285714285716e-05, 'epoch': 1.43}
{'loss': 0.7553, 'learning_rate': 2.2623809523809525e-05, 'epoch': 1.61}
{'loss': 0.7382, 'learning_rate': 2.1433333333333334e-05, 'epoch': 1.79}
{'loss': 0.7328, 'learning_rate': 2.0242857142857143e-05, 'epoch': 1.96}


...

Saving model checkpoint to ./en-kd-XLM-minilmv2-4\checkpoint-14000
Configuration saved in ./en-kd-XLM-minilmv2-4\checkpoint-14000\config.json


{'loss': 0.5453, 'learning_rate': 9.523809523809524e-09, 'epoch': 5.0}


Model weights saved in ./en-kd-XLM-minilmv2-4\checkpoint-14000\pytorch_model.bin
tokenizer config file saved in ./en-kd-XLM-minilmv2-4\checkpoint-14000\tokenizer_config.json
Special tokens file saved in ./en-kd-XLM-minilmv2-4\checkpoint-14000\special_tokens_map.json
tokenizer config file saved in ./en-kd-XLM-minilmv2-4\tokenizer_config.json
Special tokens file saved in ./en-kd-XLM-minilmv2-4\special_tokens_map.json
Deleting older checkpoint [en-kd-XLM-minilmv2-4\checkpoint-11200] due to args.save_total_limit


Training completed. Do not forget to share your model on huggingface.co/models =)




{'train_runtime': 16099.137, 'train_samples_per_second': 27.827, 'train_steps_per_second': 0.87, 'train_loss': 1.127668940952846, 'epoch': 5.0}


TrainOutput(global_step=14000, training_loss=1.127668940952846, metrics={'train_runtime': 16099.137, 'train_samples_per_second': 27.827, 'train_steps_per_second': 0.87, 'train_loss': 1.127668940952846, 'epoch': 5.0})

In [14]:
distil_trainer.push_to_hub("Training Completed")

Saving model checkpoint to ./en-kd-XLM-minilmv2-4
Configuration saved in ./en-kd-XLM-minilmv2-4\config.json
Model weights saved in ./en-kd-XLM-minilmv2-4\pytorch_model.bin
tokenizer config file saved in ./en-kd-XLM-minilmv2-4\tokenizer_config.json
Special tokens file saved in ./en-kd-XLM-minilmv2-4\special_tokens_map.json
Several commits (2) will be pushed upstream.
The progress bars may be unreliable.


Upload file pytorch_model.bin:   0%|          | 32.0k/448M [00:00<?, ?B/s]

Upload file runs/May17_14-11-13_DESKTOP-638AK67/events.out.tfevents.1652776929.DESKTOP-638AK67: 100%|#########…

remote: Enforcing permissions...        
remote: Allowed refs: all        
remote: error: cannot lock ref 'refs/heads/main': is at 2adbd8597f3d32d1698dc02da3d23ffea946462d but expected c982bafafbecac66efe82f2eb070f0b1321851d2        
To https://huggingface.co/subhasisj/en-kd-XLM-minilmv2-4
 ! [remote rejected] main -> main (failed to update ref)
error: failed to push some refs to 'https://huggingface.co/subhasisj/en-kd-XLM-minilmv2-4'



OSError: remote: Enforcing permissions...        
remote: Allowed refs: all        
remote: error: cannot lock ref 'refs/heads/main': is at 2adbd8597f3d32d1698dc02da3d23ffea946462d but expected c982bafafbecac66efe82f2eb070f0b1321851d2        
To https://huggingface.co/subhasisj/en-kd-XLM-minilmv2-4
 ! [remote rejected] main -> main (failed to update ref)
error: failed to push some refs to 'https://huggingface.co/subhasisj/en-kd-XLM-minilmv2-4'
